In [101]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy

In [102]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv")

coin_ids = df_market_data['coin_id']
df_market_data = df_market_data.set_index('coin_id')

df_market_data.head(3)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954


# Generate summary statistics
df_market_data.describe()

In [103]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [104]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_market_data)

In [105]:
# Create a DataFrame with the scaled data
column_names = df_market_data.columns
df_market_data_scaled = pd.DataFrame(scaled_data, columns=column_names)

# Set the coinid column as index
df_market_data_scaled['coin_id'] = coin_ids
df_market_data_scaled = df_market_data_scaled.set_index('coin_id')


# Display sample data
df_market_data_scaled.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [106]:
# Create an empty list to store the inertia values
k = list(range(1,12))
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
    k_model = KMeans(n_clusters=i)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)

In [107]:
# Create a dictionary with the data to plot the Elbow curve
elbow_dict = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_dict)
elbow_df.head()

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,64.127997


In [108]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow = elbow_df.hvplot.line(
    x='k',
    y='inertia',
    xticks=k,
    title='Elbow Curve using Original Scaled Data'
)

elbow

:Curve   [k]   (inertia)

Based on the elbow curve, the two best possible values for k are k=3,4. 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [109]:
# Initialize the K-Means model using the best value for k
k_model = KMeans(n_clusters=4)
k_model.fit(df_market_data_scaled)

KMeans(n_clusters=4)

In [110]:
# Predict the clusters to group the cryptocurrencies using the scaled data
predictions = k_model.predict(df_market_data_scaled)
print(predictions)

[1 1 3 3 1 1 1 1 1 3 3 3 3 1 3 1 3 3 1 3 3 1 3 3 3 3 3 3 1 3 3 3 2 1 3 3 0
 3 3 3 3]


In [111]:
# Create a copy of the DataFrame
df_market_data_predictions = df_market_data_scaled.copy()

In [112]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_predictions['ClusterGroup'] = predictions
df_market_data_predictions.head(3)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,ClusterGroup
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,1
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,1
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,3


In [113]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
k_means_original = df_market_data_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by='ClusterGroup',
    hover_cols = 'coin_id',
    title='KMeans Cluster Groups based on Original Data'
)

k_means_original

:NdOverlay   [ClusterGroup]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [114]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)
pca_model = pca.fit_transform(df_market_data_scaled)
pca_model[0:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [115]:
# Retrieve the explained variance to determine how much information can be attributed to each principal component.
pca_variance = pca.explained_variance_ratio_
pca_variance

array([0.3719856 , 0.34700813, 0.17603793])

In [116]:
variance_total = numpy.sum(pca_variance)
print(f'The total explained variance of the 3 components is: {variance_total}')

The total explained variance of the 3 components is: 0.8950316570309841


In [117]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(pca_model, columns=['price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d'])
pca_df['coin_id'] = coin_ids
pca_df = pca_df.set_index('coin_id')
pca_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [118]:
# Create an empy list to store the inertia values
k = list(range(1,12))
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
    k_model = KMeans(n_clusters=i)
    k_model.fit(pca_df)
    inertia.append(k_model.inertia_)

In [119]:
# Create a dictionary with the data to plot the Elbow curve
elbow_dict_pca = {"k":k, "inertia":inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_pca_df = pd.DataFrame(elbow_dict_pca)
elbow_pca_df.head()

,k,inertia
0,1,256.874086
1,2,168.811896
2,3,93.774626
3,4,49.665497
4,5,38.072792


In [120]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_pca = elbow_pca_df.hvplot.line(
    x='k',
    y='inertia',
    xticks=k,
    title='Elbow Curve using PCA Data'
)

elbow_pca

:Curve   [k]   (inertia)

* The best value for k when using PCA data is 4.
* In this case, k = 4 is the best value for k in both the original data and the PCA data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [121]:
# Initialize the K-Means model using the best value for k
k_model_pca = KMeans(n_clusters=4)
k_model_pca.fit(pca_df)
predictions_w_pca = k_model_pca.predict(pca_df)
predictions_w_pca[0:5]


array([3, 3, 0, 0, 3])

In [122]:
# Create a copy of the DataFrame with the PCA data
pca_df_predictions = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_df_predictions['ClusterGroup'] = predictions_w_pca
pca_df_predictions.rename(columns={'0':'a'})

# Display sample data
pca_df_predictions.head(3)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,ClusterGroup
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,3
ethereum,-0.458261,0.458466,0.952877,3
tether,-0.433070,-0.168126,-0.641752,0


In [123]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

kmeans_pca = pca_df_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by='ClusterGroup',
    hover_cols = 'coin_id',
    title='KMeans using PCA Data'
)

kmeans_pca

:NdOverlay   [ClusterGroup]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [124]:
# Composite plot to contrast the Elbow curves
elbow*elbow_pca

:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [125]:
# Composite plot to contrast the clusters
k_means_original

:NdOverlay   [ClusterGroup]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

In [126]:
kmeans_pca

:NdOverlay   [ClusterGroup]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)


*In this instance, using PCA to reduce the number of components to 3 didn't seem to make the clusters more defined than using the original scaled data, where all components were being used to cluster each coin.